Stress test:
- There are 5531452 rows in the csv file in total (including header)
- It takes 3m31s to read train_data.csv

In [41]:
# N = sum(1 for line in open('data/train_data.csv'))
# N
# print(sum(1 for line in open('data/train_labels.csv')))

458914


In [39]:
# df = pd.read_csv('data/train_data.csv')
# labels = pd.read_csv('data/train_labels.csv')

## Setup

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
features = pd.read_csv('data/train_data.csv')
labels = pd.read_csv('data/train_labels.csv')

#### Categorize columns by type: 

In [2]:
features.dtypes.unique()

array([dtype('O'), dtype('float64'), dtype('int64')], dtype=object)

In [3]:
obj_cols = features.select_dtypes(include=['O']).columns
obj_cols

Index(['customer_ID', 'S_2', 'D_63', 'D_64'], dtype='object')

In [4]:
int_cols = features.select_dtypes(include=['int64']).columns
int_cols

Index(['B_31'], dtype='object')

In [5]:
print('B_31', features.B_31.unique())
for col in obj_cols.drop(['customer_ID', 'S_2']): # S_2 is a date
    print(col, features[col].unique())

B_31 [1 0]
D_63 ['CR' 'CO' 'CL' 'XZ' 'XM' 'XL']
D_64 ['O' 'R' nan 'U' '-1']


In [6]:
float_cols = features.select_dtypes(include=['float64']).columns
float_cols

Index(['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42',
       'D_43',
       ...
       'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143',
       'D_144', 'D_145'],
      dtype='object', length=185)

In [7]:
nullcols = []
for col in features:
    if features[col].isnull().values.any():
        nullcols.append(col)
len(nullcols)

122

#### Generate distribution histograms of the columns:

In [8]:
# features.hist(bins=20, figsize=(60, 40))

## Preprocessing

In [10]:
def get_sample(size):
    prng = np.random.RandomState(0)
    g = features.groupby('customer_ID')

    portion = features[g.ngroup().isin(prng.choice(g.ngroups, size, replace=False))]
    return portion

# sample1 = get_sample(g.ngroups//100)
sample2 = get_sample(1000)

In [ ]:
# fitted_imputers = sample2.groupby('customer_ID').apply(lambda df: SimpleImputer(strategy='most_frequent').fit(df[['D_64', 'D_63']]))

# # def transform_col(df):
# #     df['D_64_trfm'] = 
# transformed = sample2.groupby('customer_ID').apply(lambda df: pd.DataFrame(fitted_imputers.loc[df['customer_ID'].iloc[0]].transform(df[['D_64', 'D_63']]), columns=[f'{name}_trfm' for name in ['D_64', 'D_63']]))
# transformed

#### Pipeline:
- Apply OHE/MEstimate
- Combine IDs
- Impute missing values
- (PCA)
- (KMeans)
- Train
- Predict

In [53]:
from lib2to3.pytree import Base
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sqlalchemy import Column

def get_y(features: pd.DataFrame):
    label_lookup = dict()
    for id in features.customer_ID:
        if id not in label_lookup:
            target = labels.loc[labels.customer_ID == id, 'target']
            label_lookup[id] = target.iloc[0]

    return features.customer_ID.apply(lambda id: label_lookup[id])
    # for id in features.customer_ID.unique():
    #     if len(features.loc[features.customer_ID == id, 'target'].unique()) != 1:
    #         print(features.loc[features.customer_ID == id])

class IDCombiner(BaseEstimator, TransformerMixin):
    def __init__(self, id_name: str):
        self.id_name = id_name
    def fit(self, X, y = None):
        return self
    def transform(self, X, y = None):
        return X.groupby(self.id_name).apply(lambda df: df.mean()).unstack().reset_index()

# class ByIDTransformer(BaseEstimator, TransformerMixin):
#     def __init__(self, id_name: str, cname: str, transformer):
#         self.id_name = id_name
#         self.cname = cname
#         self.
#     def fit(self, X: pd.DataFrame, y = None):
#         X.groupby(self.id_name)[self.cname].apply()
#         return self
#     def transform(self, X, y = None):

def column_types(features: pd.DataFrame):
    categoricals = []
    objects = []
    numericals = []
    for cname in features.columns:
        col = features[cname]
        if col.dtype == 'object':
            if col.nunique() < 10:
                categoricals.append(cname)
            else:
                objects.append(cname)
        elif col.dtype in ['int64', 'float64']:
            numericals.append(cname)
        else:
            print(f"{cname}: no column category matched")
    return categoricals, numericals, objects

def pipe(features: pd.DataFrame):
    categoricals, _, _ = column_types(features)
    
    # Preprocesses categoricals
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    cat_prep = ColumnTransformer(transformers=[
        ('cat', categorical_transformer, categoricals)
    ])

    preprep = Pipeline(steps=[
        ('cat', cat_prep),
        ('id_transformer', IDCombiner('customer_ID'))
    ])

    categoricals, numericals, _ = column_types(features)
    numerical_transformer = SimpleImputer(strategy='mean')
    num_prep = ColumnTransformer(transformers=[
        ('num', numerical_transformer, numericals)
    ])

    preprocessor = Pipeline(steps=[
        ('preprep', preprep),
        ('num', num_prep)
    ])
    
    return preprocessor

X = sample2.drop('D_64', axis=1)
df = pipe(X).fit_transform(X, get_y(X))
df.head()

AttributeError: groupby not found

Not sure how to make sense of D_64, dropping the column for now